In [19]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

# Load the dataset
file_path = 'combined_data.xlsx' # Update with the actual path
data = pd.read_excel(file_path)

# Convert sentiment labels from -1 to 0 (positive) and 1 to 1 (negative)
label_columns = ['label'] + [col for col in data.columns if col.startswith('label_')]
data[label_columns] = data[label_columns].applymap(lambda x: 0 if x == -1 else 1)

# Determine the overall sentiment
data['overall_sentiment'] = data[label_columns].max(axis=1)

# Tokenize the sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['sentence'])
sequences = tokenizer.texts_to_sequences(data['sentence'])

# Padding sequences to ensure uniform input size
max_length = max(len(seq) for seq in sequences)
X_pad = pad_sequences(sequences, maxlen=max_length, padding='post')

# Prepare target variable
y = data['overall_sentiment'].values

# Splitting dataset into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# CNN model creation
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=50, input_length=max_length),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model training
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Model summary for reference
model.summary()


Epoch 1/10
236/236 [==============================] - 18s 73ms/step - loss: 0.3422 - accuracy: 0.8889 - val_loss: 0.2043 - val_accuracy: 0.8964
Epoch 2/10
236/236 [==============================] - 18s 77ms/step - loss: 0.1839 - accuracy: 0.9055 - val_loss: 0.1684 - val_accuracy: 0.9416
Epoch 3/10
236/236 [==============================] - 17s 72ms/step - loss: 0.1374 - accuracy: 0.9549 - val_loss: 0.1699 - val_accuracy: 0.9389
Epoch 4/10
236/236 [==============================] - 17s 72ms/step - loss: 0.1057 - accuracy: 0.9704 - val_loss: 0.1851 - val_accuracy: 0.9448
Epoch 5/10
236/236 [==============================] - 18s 75ms/step - loss: 0.0860 - accuracy: 0.9788 - val_loss: 0.1929 - val_accuracy: 0.9405
Epoch 6/10
236/236 [==============================] - 17s 72ms/step - loss: 0.0729 - accuracy: 0.9849 - val_loss: 0.2195 - val_accuracy: 0.9389
Epoch 7/10
236/236 [==============================] - 17s 71ms/step - loss: 0.0622 - accuracy: 0.9887 - val_loss: 0.2247 - val_accuracy:

#**Calculate Accuracy**

In [20]:
# Evaluate the model's performance
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

print(f"Test Accuracy: {accuracy*100:.2f}%")

Test Accuracy: 94.21%


In [34]:
def predict_sentiment(text):
    # Tokenize the text
    sequence = tokenizer.texts_to_sequences([text])
    # Pad the sequence
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post')
    # Predict sentiment
    prediction = model.predict(padded_sequence)
    # Convert prediction to sentiment label
    sentiment = 'Negative' if prediction[0][0] > 0.5 else 'Positive'
    return sentiment, prediction[0][0]

# Example usage
sentence1 = "We cannot and will not be liable for any loss or damage arising from your failure to comply with the above."
sentence2 = "We may suspend or terminate your account or cease providing you with all or part of the Services"
sentence3 = "You are responsible for your use of the Services and for any Content you provide, including compliance with applicable laws, rules, and regulations."
sentiment, confidence = predict_sentiment(sentence1)
print(f"Predicted Sentiment: {sentiment} (Confidence: {confidence:.2f})")

sentiment, confidence = predict_sentiment(sentence2)
print(f"Predicted Sentiment: {sentiment} (Confidence: {confidence:.2f})")

sentiment, confidence = predict_sentiment(sentence3)
print(f"Predicted Sentiment: {sentiment} (Confidence: {confidence:.2f})")


1/1 [==============================] - 0s 21ms/step
Predicted Sentiment: Negative (Confidence: 0.74)
1/1 [==============================] - 0s 25ms/step
Predicted Sentiment: Negative (Confidence: 0.74)
1/1 [==============================] - 0s 22ms/step
Predicted Sentiment: Positive (Confidence: 0.00)


In [35]:
import nltk
from nltk.tokenize import sent_tokenize

# Ensure you've downloaded the 'punkt' tokenizer models
nltk.download('punkt')

def print_negative_sentences(paragraph):
    # Split the paragraph into sentences
    sentences = sent_tokenize(paragraph)
    # Process each sentence
    for sentence in sentences:
        sentiment, confidence = predict_sentiment(sentence)
        if sentiment == 'Negative':
            print(f"Negative Sentence: {sentence} (Confidence: {confidence:.2f})")

# Example usage
paragraph = "We may suspend or terminate your account or cease providing you with all or part of the Services at any time if we reasonably believe: (i) you have violated these Terms or our Rules and Policies, (ii) you create risk or possible legal exposure for us; (iii) your account should be removed due to unlawful conduct; (iv) your account should be removed due to prolonged inactivity; or (v) our provision of the Services to you is no longer commercially viable. We will make reasonable efforts to notify you by the email address associated with your account or the next time you attempt to access your account, depending on the circumstances. To the extent permitted by law, we may also terminate your account or cease providing you with all or part of the Services for any other reason or no reason at our convenience. In all such cases, the Terms shall terminate, including, without limitation, your license to use the Services, except that the following sections shall continue to apply: 2, 3, 5, 6, and the misuse provisions of Section 4 (“Misuse of the Services”). If you believe your account was terminated in error you can file an appeal following the steps found in our Help Center (https://help.x.com/forms/account-access/appeals). For the avoidance of doubt, these Terms survive the deactivation or termination of your account."


print_negative_sentences(paragraph)


1/1 [==============================] - 0s 25ms/step
Negative Sentence: We may suspend or terminate your account or cease providing you with all or part of the Services at any time if we reasonably believe: (i) you have violated these Terms or our Rules and Policies, (ii) you create risk or possible legal exposure for us; (iii) your account should be removed due to unlawful conduct; (iv) your account should be removed due to prolonged inactivity; or (v) our provision of the Services to you is no longer commercially viable. (Confidence: 0.74)
1/1 [==============================] - 0s 34ms/step


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


1/1 [==============================] - 0s 33ms/step
Negative Sentence: To the extent permitted by law, we may also terminate your account or cease providing you with all or part of the Services for any other reason or no reason at our convenience. (Confidence: 0.74)
1/1 [==============================] - 0s 35ms/step


In [37]:
file_path = 'Spotify.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

print_negative_sentences(text)

1/1 [==============================] - 0s 21ms/step
Negative Sentence: you acknowledge that you have read and understood the agreements , accept these agreements , and agree to be bound by them . (Confidence: 0.74)
1/1 [==============================] - 0s 30ms/step
Negative Sentence: occasionally we may , in our discretion , make changes to the agreements . (Confidence: 0.74)
1/1 [==============================] - 0s 33ms/step
Negative Sentence: in some cases , we will notify you in advance , and your continued use of the service after the changes have been made will constitute your acceptance of the changes . (Confidence: 0.74)
1/1 [==============================] - 0s 23ms/step
Negative Sentence: spotify reserves the right , in its absolute discretion , to determine your eligibility for a trial , and , subject to applicable laws , to withdraw or to modify a trial at any time without prior notice and with no liability , to the greatest extent permitted under the law . (Confidence: 0.